<a href="https://colab.research.google.com/github/karthik1946/01_c/blob/main/multivariate_and_multinomial_naive_bayes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import json

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving news_category_dataset (1).json to news_category_dataset (1).json


In [ ]:
import pandas as pd


with open('news_category_dataset (1).json', 'r') as json_file:
    lines = json_file.readlines()


data = [json.loads(line) for line in lines]


df = pd.DataFrame(data)


desired_categories = ["BUSINESS", "COMEDY", "SPORTS", "CRIME", "RELIGION", "HEALTHY LIVING", "POLITICS"]


filtered_df = df[df['category'].isin(desired_categories)]

filtered_df

,short_description,headline,date,link,authors,category
0,She left her husband. He killed their children...,There Were 2 Mass Shootings In Texas Last Week...,2018-05-26,https://www.huffingtonpost.com/entry/texas-ama...,Melissa Jeltsen,CRIME
13,Last month a Health and Human Services officia...,Trump's Crackdown On Immigrant Parents Puts Mo...,2018-05-26,https://www.huffingtonpost.com/entry/immigrant...,Elise Foley and Roque Planas,POLITICS
14,The wiretaps feature conversations between Ale...,'Trump's Son Should Be Concerned': FBI Obtaine...,2018-05-26,https://www.huffingtonpost.com/entry/fbi-wiret...,"Michael Isikoff, Yahoo News",POLITICS
15,"But don't count on Robert Mueller to nail him,...",Edward Snowden: There's No One Trump Loves Mor...,2018-05-26,https://www.huffingtonpost.com/entry/edward-sn...,Mary Papenfuss,POLITICS
16,Just a peeping minute.,Booyah: Obama Photographer Hilariously Trolls ...,2018-05-26,https://www.huffingtonpost.com/entry/obama-pho...,Mary Papenfuss,POLITICS
...,...,...,...,...,...,...
124979,,Lawmakers Introduce Bill To Tackle Misleadingl...,2014-04-18,https://www.huffingtonpost.com/entry/photoshop...,Shadee Ashtari,POLITICS
124980,"I'm assuming you are aware of this issue, Pres...",Dear President Obama,2014-04-18,https://www.huffingtonpost.com/entry/dear-pres...,"Robin Amos Kahn, ContributorWriter, Speaker, L...",POLITICS
124983,,The Obama Effect,2014-04-18,https://www.huffingtonpost.com/entry/the-potus...,,POLITICS
124986,,Police Want your Home Video Surveillance Footage,2014-04-18,https://www.huffingtonpost.com/entry/police-wa...,"Robert Siciliano, ContributorPersonal Security...",CRIME


In [ ]:
q=filtered_df[['short_description','headline','category']]
q

,short_description,headline,category
0,She left her husband. He killed their children...,There Were 2 Mass Shootings In Texas Last Week...,CRIME
13,Last month a Health and Human Services officia...,Trump's Crackdown On Immigrant Parents Puts Mo...,POLITICS
14,The wiretaps feature conversations between Ale...,'Trump's Son Should Be Concerned': FBI Obtaine...,POLITICS
15,"But don't count on Robert Mueller to nail him,...",Edward Snowden: There's No One Trump Loves Mor...,POLITICS
16,Just a peeping minute.,Booyah: Obama Photographer Hilariously Trolls ...,POLITICS
...,...,...,...
124979,,Lawmakers Introduce Bill To Tackle Misleadingl...,POLITICS
124980,"I'm assuming you are aware of this issue, Pres...",Dear President Obama,POLITICS
124983,,The Obama Effect,POLITICS
124986,,Police Want your Home Video Surveillance Footage,CRIME


In [ ]:
import pandas as pd
from collections import Counter
from sklearn.model_selection import train_test_split

# Assuming you have filtered_df with 'headline' and 'category' columns
X = filtered_df[['headline']]
y = filtered_df['category']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Combine X_train and y_train into a DataFrame
data = pd.DataFrame({'text': X_train['headline'], 'category': y_train})

# Create a dictionary to store word counts for each category
word_counts_by_category = {category: Counter() for category in data['category'].unique()}

# Tokenize and count words for each category
for category, text in zip(data['category'], data['text']):
    words = text.lower().split()  # Split text into words and convert to lowercase
    word_counts_by_category[category].update(words)

# Laplace smoothing parameter
alpha = 1.0

# Calculate class probabilities
class_probs = {}
total_docs = len(y_train)
for category, word_counts in word_counts_by_category.items():
    class_probs[category] = len(y_train[y_train == category]) / total_docs

# Predict category for each test sentence
y_pred = []
for _, row in X_test.iterrows():
    sentence = row['headline'].lower().split()  # Tokenize test sentence
    max_prob = float('-inf')
    predicted_category = None

    for category, word_counts in word_counts_by_category.items():
        prob = 1.0
        total_words_in_category = sum(word_counts.values())

        for word in sentence:
            word_count = word_counts.get(word, 0)
            word_prob = (word_count + alpha) / (total_words_in_category + alpha * len(word_counts))
            prob *= word_prob

        prob *= class_probs[category]

        if prob > max_prob:
            max_prob = prob
            predicted_category = category

    y_pred.append(predicted_category)

# Calculate accuracy
accuracy = sum(y_test == y_pred) / len(y_test)

print(f"Accuracy with Laplace Smoothing: {accuracy}")


Accuracy with Laplace Smoothing: 0.6817110432125709


In [ ]:
import pandas as pd
from collections import Counter
from sklearn.model_selection import train_test_split

X = filtered_df[['headline']]
y = filtered_df['category']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, random_state=42)

# Combine X_train and y_train into a DataFrame
data = pd.DataFrame({'text': X_train['headline'], 'category': y_train})

# Create a dictionary to store word counts for each category
word_counts_by_category = {category: Counter() for category in data['category'].unique()}

# Tokenize and count words for each category
for category, text in zip(data['category'], data['text']):
    words = text.lower().split()  # Split text into words and convert to lowercase
    word_counts_by_category[category].update(words)

# Laplace smoothing parameter
alpha = 1.0

# Calculate class probabilities
class_probs = {}
total_docs = len(y_train)
for category, word_counts in word_counts_by_category.items():
    class_probs[category] = len(y_train[y_train == category]) / total_docs

# Initialize dictionaries to store counts of actual and predicted categories
actual_counts = {category: 0 for category in y_test.unique()}
predicted_counts = {category: 0 for category in y_test.unique()}

# Predict category for each test sentence and update counts
y_pred = []
for idx, row in X_test.iterrows():
    sentence = row['headline'].lower().split()  # Tokenize test sentence
    max_prob = float('-inf')
    predicted_category = None

    for category, word_counts in word_counts_by_category.items():
        prob = 1.0
        total_words_in_category = sum(word_counts.values())

        for word in sentence:
            word_count = word_counts.get(word, 0)
            word_prob = (word_count + alpha) / (total_words_in_category + alpha * len(word_counts))
            prob *= word_prob

        prob *= class_probs[category]

        if prob > max_prob:
            max_prob = prob
            predicted_category = category

    y_pred.append(predicted_category)
    actual_category = y_test.loc[idx]
    actual_counts[actual_category] += 1
    predicted_counts[predicted_category] += 1

# Calculate accuracy
accuracy = sum(y_test == y_pred) / len(y_test)

print(f"Accuracy with Laplace Smoothing: {accuracy}")
print("Actual category counts:", actual_counts)
print("Predicted category counts:", predicted_counts)


Accuracy with Laplace Smoothing: 0.6891294227188082
Actual category counts: {'POLITICS': 4931, 'HEALTHY LIVING': 1021, 'COMEDY': 585, 'BUSINESS': 632, 'SPORTS': 625, 'CRIME': 436, 'RELIGION': 362}
Predicted category counts: {'POLITICS': 3431, 'HEALTHY LIVING': 1287, 'COMEDY': 830, 'BUSINESS': 716, 'SPORTS': 630, 'CRIME': 766, 'RELIGION': 932}


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer

X = filtered_df[[ 'headline']]
y = filtered_df['category']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

X_train

,headline
100544,Puppy-Kicking CEO Charged In Elevator Incident...
13798,#MeToo: Exposing The Heresy Of Twisting Script...
110004,Is Your Health Care on a Spiritual Path?
43543,The Planet Is Screwed If Democrats Don't Win T...
41788,Donald Trump Is Handling The Transition Exactl...
...,...
117681,Central American Minors Seek Refuge in the US:...
78319,REPORT: Biden Eyes Weekend Decision On 2016
1773,Flight Records Disprove Trump's Claim About 'P...
32178,Strong Leadership In The White House Requires ...


In [ ]:
import pandas as pd
from collections import Counter

data = pd.DataFrame({'text': X_train['headline'], 'category': y_train})

word_counts_by_category = {category: Counter() for category in data['category'].unique()}

for category, text in zip(data['category'], data['text']):
    words = set(text.lower().split())
    word_counts_by_category[category].update(words)

word_counts_df = pd.DataFrame.from_dict(word_counts_by_category, orient='index').fillna(0)

word_counts_df = word_counts_df.T

print(word_counts_df)


                 POLITICS  SPORTS  COMEDY  CRIME  HEALTHY LIVING  RELIGION  \
mocked               16.0     2.0     2.0    0.0             0.0       0.0   
1987                  1.0     0.0     0.0    0.0             0.0       0.0   
supreme             505.0     0.0     4.0    1.0             1.0       5.0   
president           500.0    17.0    41.0    2.0             8.0      14.0   
in                 3919.0   512.0   395.0  743.0           591.0     323.0   
...                   ...     ...     ...    ...             ...       ...   
birds!                0.0     0.0     1.0    0.0             0.0       0.0   
mice'                 0.0     0.0     1.0    0.0             0.0       0.0   
giraffe               0.0     0.0     1.0    0.0             0.0       0.0   
geoffrey              0.0     0.0     1.0    0.0             0.0       0.0   
#manafortmonday       0.0     0.0     1.0    0.0             0.0       0.0   

                 BUSINESS  
mocked                0.0  
1987   

In [ ]:
import pandas as pd
from collections import Counter
from sklearn.model_selection import train_test_split


data = pd.DataFrame({'text': X_train['headline'], 'category': y_train})

# Create a dictionary to store unique word counts for each category
unique_word_counts_by_category = {category: Counter() for category in data['category'].unique()}

# Tokenize and count unique words for each category
for category, text in zip(data['category'], data['text']):
    words = set(text.lower().split())  # Split text into words, convert to lowercase, and keep only unique words
    unique_word_counts_by_category[category].update(words)

# Laplace smoothing parameter
alpha = 1.0

# Calculate class probabilities
class_probs = {}
total_docs = len(y_train)
for category, unique_word_counts in unique_word_counts_by_category.items():
    class_probs[category] = len(y_train[y_train == category]) / total_docs

# Initialize dictionaries to store counts of actual and predicted categories
actual_counts = {category: 0 for category in y_test.unique()}
predicted_counts = {category: 0 for category in y_test.unique()}

# Predict category for each test sentence and update counts
y_pred = []
for idx, row in X_test.iterrows():
    sentence = set(row['headline'].lower().split())  # Tokenize test sentence, convert to lowercase, and keep only unique words
    max_prob = float('-inf')
    predicted_category = None

    for category, unique_word_counts in unique_word_counts_by_category.items():
        prob = 1.0
        total_unique_words_in_category = len(unique_word_counts)

        for word in sentence:
            word_count = unique_word_counts.get(word, 0)
            word_prob = (word_count + alpha) / (total_unique_words_in_category + alpha * len(unique_word_counts))
            prob *= word_prob

        prob *= class_probs[category]

        if prob > max_prob:
            max_prob = prob
            predicted_category = category

    y_pred.append(predicted_category)
    actual_category = y_test.loc[idx]
    actual_counts[actual_category] += 1
    predicted_counts[predicted_category] += 1

accuracy = sum(y_test == y_pred) / len(y_test)

print(f"Accuracy with Laplace Smoothing: {accuracy}")
print("Actual category counts:", actual_counts)
print("Predicted category counts:", predicted_counts)


Accuracy with Laplace Smoothing: 0.7589618249534451
Actual category counts: {'POLITICS': 4931, 'HEALTHY LIVING': 1021, 'COMEDY': 585, 'BUSINESS': 632, 'SPORTS': 625, 'CRIME': 436, 'RELIGION': 362}
Predicted category counts: {'POLITICS': 6325, 'HEALTHY LIVING': 1010, 'COMEDY': 231, 'BUSINESS': 141, 'SPORTS': 356, 'CRIME': 407, 'RELIGION': 122}


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split


# Count the frequency of each category in the training set
train_category_counts = y_train.value_counts()

# Count the frequency of each category in the test set
test_category_counts = y_test.value_counts()

# Print the counts for each category in the training set
print("Training Set Category Counts:")
print(train_category_counts)

# Print the counts for each category in the test set
print("\nTest Set Category Counts:")
print(test_category_counts)


Training Set Category Counts:
POLITICS          26173
HEALTHY LIVING     5345
BUSINESS           3399
SPORTS             3339
COMEDY             3178
CRIME              2313
RELIGION           2072
Name: category, dtype: int64

Test Set Category Counts:
POLITICS          6566
HEALTHY LIVING    1349
BUSINESS           855
SPORTS             828
COMEDY             793
CRIME              580
RELIGION           484
Name: category, dtype: int64
